In [1]:
import requests
import pandas as pd
import os
import csv
import scipy.stats as stats
import numpy as np
import math
from scipy.stats import shapiro 
from scipy.stats import lognorm

In [2]:

session = requests.Session()
session.auth = ('180dc', 'WinterTerm!!')

res2 = session.get('https://api.ona.io/api/v1/export') 

get_data = res2.json()

print(get_data)
data_availible=[]
Land_data=[]
PCQ_data=[]
inde = 0
for d in get_data:
  inp_data = []
  inp_data.append(d.get('filename'))
  inp_data.append(d.get('export_url'))
  data_availible.append(inp_data)
  if inp_data[0][0:3] == 'GC6':
    if inp_data[0][4:8] == 'Land':
      Land_data.append([d.get('filename'), inde])
    else: 
      PCQ_data.append([d.get('filename'), inde])
    inde+=1

breakpoint()

print('There are ', len(Land_data), ' land restoration files and ', len(PCQ_data), ' PCQ data files')
print('Please select a PCQ file from: ')
index = 1
for d in PCQ_data:
  print(index,' = ',d[0])
  index+=1
num_pcq = int(input('Index: '))
print('Please select a Land resotration file from: ')
index = 1
for d in Land_data:
  print(index,' = ',d[0])
  index+=1

num_land = int(input('Index: '))
PCQ_download = PCQ_data[num_pcq-1][0]
Land_download = Land_data[num_land-1][0]

Land_res_fileName = Land_download # Original test case: 'GC6_Land_Restoration_Survey_Oct-Dec21_1_2022_11_06_13_55_35_124329.csv' 
PCQ_fileName = PCQ_download # Original test case:'GC6_PCQ_Survey_Apr-Jun22_FINAL_1_2022_11_04_10_49_03_971434.csv'

Land_res=[]
PCQ=[]

for i in data_availible:
  if Land_res_fileName in i:
    with requests.Session() as s:
      s.auth = ('180dc', 'WinterTerm!!')
      download = s.get(i[1])
      decoded_content = download.content.decode('utf-8')
      cr = csv.reader(decoded_content.splitlines(), delimiter=',')
      my_list = list(cr)
      for row in my_list:
          Land_res.append(row)
  if PCQ_fileName in i:
    with requests.Session() as s:
      s.auth = ('180dc', 'WinterTerm!!')
      download = s.get(i[1])
      decoded_content = download.content.decode('utf-8')
      cr = csv.reader(decoded_content.splitlines(), delimiter=',')
      my_list = list(cr)
      for row in my_list:
          PCQ.append(row)
      

PCQ = pd.DataFrame(data=PCQ[1:],
                    columns=PCQ[0])
Land_res = pd.DataFrame(data=Land_res[1:],
                    columns=Land_res[0])


[{'id': 2674179, 'job_status': 'SUCCESS', 'type': 'csv', 'task_id': '818b5c95-76e3-411c-87ff-ae5d8e2983c4', 'xform': 'https://api.ona.io/api/v1/forms/706189', 'date_created': '2023-01-25T15:24:07.288905Z', 'filename': 'GC6_Land_Restoration_Survey_Oct-Dec21_1_2023_01_25_15_24_11_804048.csv', 'options': {'language': 'English (en)', 'include_hxl': True, 'include_images': False, 'include_labels': False, 'win_excel_utf8': False, 'group_delimiter': '/', 'include_reviews': False, 'remove_group_name': False, 'include_labels_only': False, 'split_select_multiples': False}, 'export_url': 'https://api.ona.io/api/v1/export/2674179.csv', 'error_message': None}, {'id': 2674180, 'job_status': 'SUCCESS', 'type': 'csv', 'task_id': '76eee1c8-1f10-46a6-902c-fabf48524fd8', 'xform': 'https://api.ona.io/api/v1/forms/706180', 'date_created': '2023-01-25T15:24:37.766220Z', 'filename': 'GC6_PCQ_Survey_Apr-Jun22_FINAL_1_2023_01_25_15_24_40_556415.csv', 'options': {'language': 'English (en)', 'include_hxl': True,

In [3]:
class data_cleaning:

  def __init__(self, df):
    self.df = df

  def view_df(self, value = 10):
    '''view the data frame'''
    self.value = value
    return self.df.head(value)#self.df.to_string()


  def remove_empty_columns(self):
    '''the purpose of this function is to 
    remove columns with null values'''
    self.df.dropna(how='all', axis=1, inplace=True)
    return self.df.shape
    

  def display_repeats(self):
    '''function to find repeating 
    columns that end with point .1'''

    repeats = []
    self.repeats = repeats
    for column in list(self.df.columns):
      
      if column.endswith('.1'):
        self.repeats.append(column)
    return self.repeats

   
# This function is not required as pandas adds a .1 a duplicate column name 

  def display_repeating_columns1(self):
    ''' purpose of this function is 
    to create and display an array of 
    all the duplicated columns with the 
    same column name
    '''
    repeated_columns1 = []
    self.repeated_columns1 = repeated_columns1
    list_of_all_columns = list(self.df.columns)
    for column in list_of_all_columns:
        if list_of_all_columns.count(column) > 1:
            self.repeated_columns1.append(column)
    return self.repeated_columns1
  
  def display_repeating_columns2(self):

    '''purpose of this function is to find 
    columns with the same values and pair
    these columns up'''

    repeated_columns2 = []
    self.repeated_columns2 = repeated_columns2
    
    for x in range(self.df.shape[1]):
        col = self.df.iloc[:, x]
        for y in range(x + 1, self.df.shape[1]):
            othercol = self.df.iloc[:, y]
            if col.equals(othercol):
              self. repeated_columns2.append([self.df.columns[y], self.df.columns[x]])
    return self.repeated_columns2

  def deleterepeats(self):
    '''purpose of this function is to delete columns that 
    end with .1 and are therefore repeating columns as 
    panadas adds the .1 to columns with the same name'''
    
    for col in self.repeats:
      self.df = self.df.drop([col], axis = 1)
    return self.df.shape

# not needed for the same reason why the function display columns is not neeeded

  def deleterepeatingcolumns1(self):
      '''purpose of this function is 
      to delete all the duplicated columns
      with the same column name '''
      for col in self.repeated_columns1:
        self.df = self.df.drop([col], axis = 1)
      return self.df.shape

  def deleterepeatingcolumns2(self):
      ''' purpose of this function is 
      to delete all the duplicated columns
      with the same values'''
      for col in self.repeated_columns2:
        if col[0] in self.df.columns:
          self.df = self.df.drop([col[0]], axis =1 )
      return self.df

  def confidence_intervals(self):
    ''' purpose of this function is to find
    the confidence intervals of the numeric
    columns. The function will then drop rows if it 
    contains a value outside of the confidence interval
    for that column '''
    for i in range(self.df.shape[1]):
      try: 
        self.df.iloc[:, i] = self.df.iloc[:, i].apply(lambda x: float(x))
        if shapiro(self.df.iloc[:, i])[1] < 0.05:
          ci = stats.t.interval(alpha=0.99, df=self.df.shape[0], loc=np.mean(self.df.iloc[:, i]), scale=stats.sem(self.df.iloc[:, i]))
          for j in range(self.df.shape[0]):
            if (self.df.iloc[j, i] < ci[0]) or (self.df.iloc[j, i] > ci[1]):
              #print(df.columns[i])
              self.df.drop(j, inplace = True)        
      except:
        pass
    return self.df.shape

  def removing_prefix_land(self):
    '''purpose of the function is to remove 
    prefixes on columns in land restoration'''

    for col in self.df.columns:
        if col[0:13] == 'group_survey/':
          self.df.columns = self.df.columns.str.replace(col, col[13:])

    for col in self.df.columns:
        if col[0:10] =='group_gps/':
          self.df.columns = self.df.columns.str.replace(col, col[10:])

  def removing_prefix_pcq(self):
    '''purpose of the function is to remove 
    prefixes on columns in pcq'''

    for col in self.df.columns:
        if col[0:11] =='pcq_return/':
          self.df.columns = self.df.columns.str.replace(col, col[11:])

    for col in self.df.columns:
        if col[0:13] =='pcq_loop_one/':
          self.df.columns = self.df.columns.str.replace(col, col[13:])

    for col in self.df.columns:
        if col[0:19] == 'intro_metadata_grp/':
          self.df.columns = self.df.columns.str.replace(col, col[19:])
    for col in self.df.columns:
        if col[0:13] == 'metadata_grp/':
          self.df.columns = self.df.columns.str.replace(col, col[13:])
    for col in self.df.columns:
        if col[0:17] == 'geo_location_grp/':
          self.df.columns = self.df.columns.str.replace(col, col[17:])
    for col in self.df.columns:
        if col[0:20] == 'geo_location_grp_02/':
          self.df.columns = self.df.columns.str.replace(col, col[20:])
    for col in self.df.columns:
        if col[0:5] == 'meta/':
          self.df.columns = self.df.columns.str.replace(col, col[5:])

In [4]:
pcq = data_cleaning(PCQ)
pcq.remove_empty_columns()
pcq.display_repeats() # removes pandas suffix
pcq.deleterepeats()
pcq.display_repeating_columns2()
pcq.deleterepeatingcolumns2()
pcq.removing_prefix_pcq()

land = data_cleaning(Land_res)
land.remove_empty_columns()
land.display_repeats() # removes pandas suffix
land.deleterepeats()
#land.confidence_intervals()
land.display_repeating_columns2()
land.deleterepeatingcolumns2()
land.removing_prefix_land()

C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[11:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[11:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[11:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[11:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value

C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[11:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[11:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[11:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[11:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value

C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[11:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[11:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[11:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[11:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value

C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[11:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[11:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[11:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[11:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value

C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[11:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[11:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[11:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[11:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:130: FutureWarning: The default value

...

C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[13:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[13:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[13:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[13:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value

C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[13:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[13:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[13:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[13:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value

C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[13:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[13:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[13:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[13:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value

C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[13:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[13:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[13:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[13:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value

C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[13:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[13:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[13:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df.columns = self.df.columns.str.replace(col, col[13:])
C:\Users\footb\AppData\Local\Temp\ipykernel_7528\1637813330.py:118: FutureWarning: The default value